En SQL, Cascade Delete Relationships es una característica que asegura que, cuando se elimina un registro en una tabla "principal", todos los registros relacionados en la tabla "hija" también se eliminen automáticamente. Esto es útil para mantener la integridad referencial y evitar datos huérfanos (registros en la tabla hija que ya no tienen una relación válida con un registro en la tabla principal).

In [ ]:
from sqlmodel import SQLModel, Field, create_engine, Session, Relationship
from typing import List, Optional
from datetime import date

class Measurement(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    experiment_id: int = Field(foreign_key="experiments.id", nullable=False)
    temperature: float  # Temperatura en Kelvin
    pressure: float     # Presión en Pascales
    volume: float       # Volumen en m³
    experiment: "Experiment" = Relationship(back_populates="measurements")

class Experiment(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    description: Optional[str]
    date_performed: date
    measurements: List[Measurement] = Relationship(
        back_populates="experiment",
        sa_relationship_kwargs={"cascade": "all, delete-orphan"}
    )

# Crear la base de datos
engine = create_engine("sqlite:///thermodynamics.db", echo=True)
SQLModel.metadata.create_all(engine)

# Ejemplo de inserción y eliminación
with Session(engine) as session:
    # Crear un experimento con mediciones
    experiment = Experiment(
        name="Compresión adiabática",
        description="Estudio de la compresión adiabática de un gas ideal",
        date_performed="2024-11-01",
        measurements=[
            Measurement(temperature=300.15, pressure=202650, volume=0.05),
            Measurement(temperature=400.15, pressure=405300, volume=0.025)
        ]
    )
    session.add(experiment)
    session.commit()

    # Eliminar el experimento y las mediciones asociadas
    session.delete(experiment)
    session.commit()